In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 542M
6.4M  twic1417.pgn
6.8M  twic1417_moves.csv
 32K  twic1417_game_info.csv
4.5M  twic1407.pgn
 13M  twic1407_moves.csv
 72K  twic1407_game_info.csv
 16K  README.md
 89M  lichess_db_standard_rated_2013-01.pgn
286M  lichess_db_standard_rated_2013-01_moves.csv
 33M  lichess_db_standard_rated_2013-01_game_info.csv
568K  chess.ipynb
184K  chess-and-eigenvectors.ipynb
4.0K  caruana_carlsen_2018.pgn
160K  caruana_carlsen_2018_moves.csv
4.0K  caruana_carlsen_2018_game_info.csv
6.7M '2021-09.bare.[6516].pgn'
 43M '2021-09.bare.[6516]_moves.csv'
136K '2021-09.bare.[6516]_game_info.csv'
 11M '2021-01.bare.[10357].pgn'
 44M '2021-01.bare.[10357]_moves.csv'
152K '2021-01.bare.[10357]_game_info.csv'


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [ ]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    print("en passant")
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        print("checkmate")
        #do checkmate stuff here

    game_counter += 1
    print('end of game',game_counter)
    
    total_move_counter += counter
    print('total moves analyzed',total_move_counter)

    
    print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

checkmate
end of game 1
total_moves_analyzed 25
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 2
total_moves_analyzed 60
board outcome None
end of game 3
total_moves_analyzed 81
board outcome None
end of game 4
total_moves_analyzed 175
board outcome None
end of game 5
total_moves_analyzed 221
board outcome None
end of game 6
total_moves_analyzed 284
board outcome None
end of game 7
total_moves_analyzed 322
board outcome None
end of game 8
total_moves_analyzed 377
board outcome None
end of game 9
total_moves_analyzed 450
board outcome None
end of game 10
total_moves_analyzed 540
board outcome None
checkmate
end of game 11
total_moves_analyzed 614
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 12
total_moves_analyzed 707
board outcome None
en passant
end of game 13
total_moves_analyzed 756
board outcome None
checkmate
end of game 14
total_moves_analyzed 825
board outcome Outcome(termination=<Termination.CHEC

checkmate
end of game 108
total_moves_analyzed 6780
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 109
total_moves_analyzed 6919
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 110
total_moves_analyzed 7004
board outcome None
end of game 111
total_moves_analyzed 7064
board outcome None
checkmate
end of game 112
total_moves_analyzed 7129
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 113
total_moves_analyzed 7189
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 114
total_moves_analyzed 7306
board outcome None
end of game 115
total_moves_analyzed 7351
board outcome None
checkmate
end of game 116
total_moves_analyzed 7424
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 117
total_moves_analyzed 7481
board outcome None
checkmate
end of game 118
total_moves_analyzed 7537
board 

total_moves_analyzed 13011
board outcome None
end of game 209
total_moves_analyzed 13050
board outcome None
end of game 210
total_moves_analyzed 13122
board outcome None
end of game 211
total_moves_analyzed 13190
board outcome None
end of game 212
total_moves_analyzed 13229
board outcome None
end of game 213
total_moves_analyzed 13262
board outcome None
end of game 214
total_moves_analyzed 13340
board outcome None
end of game 215
total_moves_analyzed 13383
board outcome None
end of game 216
total_moves_analyzed 13420
board outcome None
end of game 217
total_moves_analyzed 13485
board outcome None
checkmate
end of game 218
total_moves_analyzed 13541
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 219
total_moves_analyzed 13582
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 220
total_moves_analyzed 13605
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
e

checkmate
end of game 312
total_moves_analyzed 19627
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 313
total_moves_analyzed 19670
board outcome None
checkmate
end of game 314
total_moves_analyzed 19752
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 315
total_moves_analyzed 19786
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 316
total_moves_analyzed 19813
board outcome None
end of game 317
total_moves_analyzed 19846
board outcome None
end of game 318
total_moves_analyzed 19901
board outcome None
en passant
end of game 319
total_moves_analyzed 19964
board outcome None
end of game 320
total_moves_analyzed 19994
board outcome None
checkmate
end of game 321
total_moves_analyzed 20053
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 322
total_moves_analyzed 20090
board outcome None
checkmate
end of game 323
total_moves_an

end of game 434
total_moves_analyzed 26267
board outcome None
end of game 435
total_moves_analyzed 26279
board outcome None
end of game 436
total_moves_analyzed 26332
board outcome None
end of game 437
total_moves_analyzed 26366
board outcome None
end of game 438
total_moves_analyzed 26428
board outcome None
checkmate
end of game 439
total_moves_analyzed 26488
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 440
total_moves_analyzed 26529
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 441
total_moves_analyzed 26577
board outcome None
end of game 442
total_moves_analyzed 26621
board outcome None
end of game 443
total_moves_analyzed 26664
board outcome None
checkmate
end of game 444
total_moves_analyzed 26766
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 445
total_moves_analyzed 26826
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, win

...

end of game 10258
total_moves_analyzed 669086
board outcome None
end of game 10259
total_moves_analyzed 669134
board outcome None
end of game 10260
total_moves_analyzed 669236
board outcome None
checkmate
end of game 10261
total_moves_analyzed 669263
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10262
total_moves_analyzed 669355
board outcome None
checkmate
end of game 10263
total_moves_analyzed 669494
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10264
total_moves_analyzed 669546
board outcome None
end of game 10265
total_moves_analyzed 669569
board outcome None
end of game 10266
total_moves_analyzed 669584
board outcome None
end of game 10267
total_moves_analyzed 669685
board outcome None
end of game 10268
total_moves_analyzed 669789
board outcome None
end of game 10269
total_moves_analyzed 669858
board outcome None
end of game 10270
total_moves_analyzed 669974
board outcome None
end of game 10271
total

checkmate
end of game 10360
total_moves_analyzed 676143
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 10361
total_moves_analyzed 676210
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10362
total_moves_analyzed 676268
board outcome None
end of game 10363
total_moves_analyzed 676328
board outcome None
end of game 10364
total_moves_analyzed 676413
board outcome None
checkmate
end of game 10365
total_moves_analyzed 676496
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 10366
total_moves_analyzed 676560
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 10367
total_moves_analyzed 676626
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 10368
total_moves_analyzed 676726
board outcome None
end of game 10369
total_moves_analyzed 676816
board outcome None
end of game 10370


total_moves_analyzed 683282
board outcome None
end of game 10464
total_moves_analyzed 683360
board outcome None
end of game 10465
total_moves_analyzed 683406
board outcome None
end of game 10466
total_moves_analyzed 683506
board outcome None
end of game 10467
total_moves_analyzed 683547
board outcome None
checkmate
end of game 10468
total_moves_analyzed 683622
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 10469
total_moves_analyzed 683694
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 10470
total_moves_analyzed 683742
board outcome None
end of game 10471
total_moves_analyzed 683796
board outcome None
checkmate
end of game 10472
total_moves_analyzed 683816
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 10473
total_moves_analyzed 683881
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10474
total_moves_analyze

end of game 10568
total_moves_analyzed 690839
board outcome None
end of game 10569
total_moves_analyzed 690898
board outcome None
end of game 10570
total_moves_analyzed 691043
board outcome None
end of game 10571
total_moves_analyzed 691195
board outcome Outcome(termination=<Termination.STALEMATE: 2>, winner=None)
checkmate
end of game 10572
total_moves_analyzed 691260
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10573
total_moves_analyzed 691280
board outcome None
end of game 10574
total_moves_analyzed 691291
board outcome None
end of game 10575
total_moves_analyzed 691343
board outcome None
end of game 10576
total_moves_analyzed 691450
board outcome None
checkmate
end of game 10577
total_moves_analyzed 691482
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 10578
total_moves_analyzed 691601
board outcome None
end of game 10579
total_moves_analyzed 691697
board outcome None
end of game 10580
total_moves_a

end of game 10677
total_moves_analyzed 698294
board outcome None
end of game 10678
total_moves_analyzed 698367
board outcome None
end of game 10679
total_moves_analyzed 698434
board outcome None
checkmate
end of game 10680
total_moves_analyzed 698481
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10681
total_moves_analyzed 698506
board outcome None
end of game 10682
total_moves_analyzed 698581
board outcome None
end of game 10683
total_moves_analyzed 698611
board outcome None
end of game 10684
total_moves_analyzed 698672
board outcome None
checkmate
end of game 10685
total_moves_analyzed 698753
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10686
total_moves_analyzed 698843
board outcome None
checkmate
end of game 10687
total_moves_analyzed 698894
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 10688
total_moves_analyzed 698946
board outcome None
end of game 10689
tota

In [ ]:
print(game_counter,'games analyzed')
print(counter,'moves analyzed')

In [ ]:
weights

In [ ]:
a = weights - np.identity(32)
print(a)

In [ ]:
b = np.zeros(32)
print(b)

In [ ]:
x = linalg.solve(a, b)
x

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [ ]:
np.linalg.det(a)

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved with a Matrix Factorization.

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(a)  

In [ ]:
values = eigen_vectors[:, np.argmin(eigen_values)] 

In [ ]:
## Let's normalize the values

In [ ]:
list = []
for i in values:
    list.append(abs(float(i)))

In [ ]:
#The fractional inverse of the strongest pawn
factor = 1 / np.max(list[8:-8])

In [ ]:
normed = [i*factor for i in list]

In [ ]:
normalized_list = ["%.2f"%i for i in normed]

In [ ]:
print(normalized_list[0:8])
print(normalized_list[8:8+8])
print(normalized_list[8+8:8+8+8])
print(normalized_list[-8:])

In [ ]:
board.reset()
game = chess.pgn.read_game(pgn)

for move in game.mainline_moves():
        print(move.uci())
        print('turn?' , board.turn)
        print('is game over?' , board.is_game_over())
        print('from square: ' ,chess.square_name(move.from_square))
        print('to square: ' ,chess.square_name(move.to_square))
        print('drop: ' ,move.drop)
        print('is capture?' , board.is_capture(move))
        print('is en passant capture?' , board.is_en_passant(move))
        print('is kingside castle?' , board.is_kingside_castling(move))
        print('is quenside castle?' , board.is_queenside_castling(move))

        board.push(move)  # Make the move
        print('is_checkmate?' , board.is_checkmate())
        #print('checkers' , board.checkers())
        print('outcome' , board.outcome())

        #print(board)
        #print(board.piece_map())
    #board